### Imports:

In [1]:
import numpy as np
import random
import data_generation
import util

### Generating Data:
* Define $ r \sim U(-50,50) $, $r_n \sim N(0,1)$, ground truth weight vector $w$ 
* Data: $x = r + r_n$
* Labels: $y = w' \cdot x$

In [2]:
w = np.random.uniform(low=-1.0, high=127/128.0, size=(20,))
n = 10000
x,y = data_generation.generate_data_linear_regression(w,n)

### Implementing Regular SGD:

In [3]:
weights_sgd = util.fp_sgd_lr(0.00001, x, y, 1000000)
print(weights_sgd)

[-0.30668232 -0.50894321 -0.5693403   0.0311398   0.12628575 -0.47267037
  0.56628577  0.0971656  -0.93116848 -0.83738968 -0.45804884 -0.35939695
 -0.3486919  -0.65336463  0.35105909 -0.95176203 -0.50926658  0.86233823
 -0.54480864 -0.75668643]


In [4]:
w

array([-0.30226339, -0.50927717, -0.56612027,  0.03591324,  0.12766173,
       -0.47496517,  0.56681217,  0.09978996, -0.92972767, -0.8343348 ,
       -0.46528375, -0.35658782, -0.34463567, -0.65263679,  0.34697617,
       -0.94908189, -0.50734389,  0.86260297, -0.54775591, -0.75156291])

### Implementing Regular SVRG:

In [5]:
weights_svrg = util.fp_svrg_lr(0.00001, x, y, 10)
print(weights_svrg)

[-0.30440511 -0.51019531 -0.56620999  0.0321444   0.12410021 -0.47568455
  0.5686342   0.10083707 -0.92987683 -0.83287081 -0.4675878  -0.35741589
 -0.34594025 -0.65433788  0.34607422 -0.94848033 -0.50726579  0.86140852
 -0.54704204 -0.75134653]


### LP SGD:

In [6]:
weights_sgd_lp = util.lp_sgd_lr(0.00001, x, y, 1000000)
print(weights_sgd_lp)
print(weights_sgd_lp*(1/128.0))

[ -6 -17 -35   1   9 -46  36   1 -95 -94 -27 -24   2 -50  30 -79 -14  84
 -34 -61]
[-0.046875  -0.1328125 -0.2734375  0.0078125  0.0703125 -0.359375   0.28125
  0.0078125 -0.7421875 -0.734375  -0.2109375 -0.1875     0.015625  -0.390625
  0.234375  -0.6171875 -0.109375   0.65625   -0.265625  -0.4765625]


### LP SVRG:

In [8]:
weights_svrg_lp = util.lp_svrg_lr(0.00001, x, y, 40)
print(weights_svrg_lp)
print(weights_svrg_lp*(1/128.0))

[-15 -32 -43   1   0 -34  41   1 -86 -86 -28 -14 -10 -47  17 -81 -34  87
 -41 -70]
[-0.1171875 -0.25      -0.3359375  0.0078125  0.        -0.265625
  0.3203125  0.0078125 -0.671875  -0.671875  -0.21875   -0.109375  -0.078125
 -0.3671875  0.1328125 -0.6328125 -0.265625   0.6796875 -0.3203125
 -0.546875 ]
